In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import load_dotenv

from utils import get_season, nfl_weeks_pull, game_keys_pull
from yahoo_query import league_season_data

# TODAY = np.datetime64("today", "D")
TODAY = np.datetime64("2021-09-28")
# TODAY = np.datetime64("2020-09-28")
# TODAY = np.datetime64("2019-09-28")
# TODAY = np.datetime64("2018-09-28")
# TODAY = np.datetime64("2017-09-28")
# TODAY = np.datetime64("2016-09-28")
# TODAY = np.datetime64("2015-09-28")
# TODAY = np.datetime64("2014-09-28")
# TODAY = np.datetime64("2013-09-28")
# TODAY = np.datetime64("2012-09-28")

load_dotenv()

PATH = Path.cwd().parents[0]

NFL_WEEKS = nfl_weeks_pull()
GAME_KEYS = game_keys_pull(first="no")
# SEASON = get_season()
SEASON = TODAY.astype("datetime64[Y]").item().year
LEAGUE_ID = GAME_KEYS[GAME_KEYS["season"] == SEASON]["league_ID"].values[0]
GAME_ID = GAME_KEYS[GAME_KEYS["season"] == SEASON]["game_id"].values[0]
nfl_weeks_list = list(NFL_WEEKS["week"][NFL_WEEKS["game_id"] == GAME_ID])

CONSUMER_KEY = os.getenv("yahoo_client_id")
CONSUMER_SECRET = os.getenv("yahoo_client_secret")

[2022-07-01 15:39:38,301 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <pyodbc.Connection object at 0x000001916A0F3DF0>
[2022-07-01 15:39:38,302 INFO] [sqlalchemy.engine.Engine.base._execute_context] SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
[2022-07-01 15:39:38,302 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] ()
[2022-07-01 15:39:38,303 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('',)
[2022-07-01 15:39:38,303 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('15.0.2000.5',)
[2022-07-01 15:39:38,304 INFO] [sqlalchemy.engine.Engine.base._execute_context] SELECT schema_name()
[2022-07-01 15:39:38,305 INFO] [sqlalchemy.engine.Engine.base._execute_context] [generated in 0.00044s] ()
[2022-07-01 15:39:38,305 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('',)
[2022-07-01 15:39:38,305 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('dbo',)
[2022-07-01 15:39:38,309 INFO] [sqlalchemy.eng

NFLWeeks successfully pulled from SQL server.


[2022-07-01 15:39:39,746 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <pyodbc.Connection object at 0x000001916A598030>
[2022-07-01 15:39:39,746 INFO] [sqlalchemy.engine.Engine.base._execute_context] SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
[2022-07-01 15:39:39,747 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] ()
[2022-07-01 15:39:39,747 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('',)
[2022-07-01 15:39:39,748 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('15.0.2000.5',)
[2022-07-01 15:39:39,748 INFO] [sqlalchemy.engine.Engine.base._execute_context] SELECT schema_name()
[2022-07-01 15:39:39,749 INFO] [sqlalchemy.engine.Engine.base._execute_context] [generated in 0.00038s] ()
[2022-07-01 15:39:39,749 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('',)
[2022-07-01 15:39:39,750 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('dbo',)
[2022-07-01 15:39:39,750 INFO] [sqlalchemy.eng

GameKeys successfully pulled from SQL server.


In [2]:
league = league_season_data(
    auth_dir=PATH,
    league_id=LEAGUE_ID,
    game_id=GAME_ID,
    game_code="nfl",
    offline=False,
    all_output_as_json=False,
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    browser_callback=True,
)
# game_keys = league.all_game_keys()
# nfl_weeks = league.all_nfl_weeks()

# meta = league.metadata(first_time=first_time)
# settings, roster, stat_cat = league.set_roster_pos_stat_cat(first_time=first_time)
# draft = league.draft_results(first_time=first_time)
# teams = league.teams_and_standings(first_time="no")

# for week in nfl_weeks_list[:-1]:
#     matchups = league.matchups_by_week(first_time='no', nfl_week=week)

# players = league.players_list(first_time="no")
# players.to_csv(f"players_{str(SEASON)}.csv", index=False)

# for week in nfl_weeks_list[:-1]:
#     week_roster = league.team_roster_by_week(first_time='no', nfl_week=week)

for week in nfl_weeks_list[:-1]:
    week_stats = league.player_stats_by_week(first_time="no", nfl_week=week)

pd.set_option("display.max_columns", 999)
week_stats

Error with player stats at player_key 406.p.28477 and week 1.
Yahoo data unavailable due to rate limiting. Please try again later.
Retrying in 30 minutes.
